In [4]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [3]:
path = r"C:\\Users\\USER\\Desktop\\font_original" #원본 이미지 경로 
copy_path = r"C:\\Users\\USER\\Desktop\\font_classifier" #복사 및 분류된 이미지 경로
#font_classifier 안에 모든 글꼴 이미지들이 다 저장되어 있음 (따로 하위폴더 없음)
#노트북 경로

### 데이터셋 준비

In [19]:
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.5,.2,.3))

Copying files: 55860 files [03:07, 298.31 files/s]


In [5]:
print(len(os.listdir(copy_path + '\\val')))
print(len(os.listdir(copy_path + '\\train')))
print(len(os.listdir(copy_path + '\\test')))

5
5
5


### 데이터셋 정형화

In [6]:
trainDataGen = ImageDataGenerator(rescale=1./255)

In [7]:
currentPath = copy_path + '\\train'

In [8]:
trainGenSet = trainDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 27930 images belonging to 5 classes.


-------------

In [9]:
valDataGen = ImageDataGenerator(rescale=1./255)

In [10]:
currentPath = copy_path + '\\val'

In [11]:
valGenSet = valDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 11170 images belonging to 5 classes.


---------------------

In [12]:
testDataGen = ImageDataGenerator(rescale=1./255)

In [13]:
currentPath = copy_path + '\\test'

In [14]:
testGenSet = testDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 16760 images belonging to 5 classes.


--------

### 모델 구성

In [15]:
# 모델 구성
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
    
model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))  #18로 되어있었는데, 출력 라벨을 바꿔야 할 거 같은 구글링 결과가 나와서 바꿔봄...! => 성공!!! (validation, train의 라벨이 다 같아야 함!!)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

### 모델 훈련

In [17]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
results = model.fit(
    trainGenSet,
    steps_per_epoch=40,
    epochs=100,
    validation_data=valGenSet,
    validation_steps=20
)

Epoch 1/100
40/40 [==============================] - 18s 316ms/step - loss: 1.5389 - accuracy: 0.3008 - val_loss: 0.8798 - val_accuracy: 0.7055
Epoch 2/100
15/40 [==========>...................] - ETA: 4s - loss: 0.7299 - accuracy: 0.6313

KeyboardInterrupt: 

### 모델 accuracy, loss 그래프로 표현

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 모델 테스트 및 정확도 출력 

In [ ]:
scores = model.evaluate_generator(testGenSet)
print(scores)